In [50]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path

Ponovno preberemo model Recon3D, H-GEM:

In [51]:
model = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

In [52]:
model_H_gem = cobra.io.read_sbml_model(os.path.join('models','Human-GEM.xml'))

In [53]:
print(model.genes[:5])
print(model_H_gem.genes[:5])

# Observe methods for model_H_gem.genes
print(dir(model_H_gem.genes[0]))

print(model_H_gem.genes[0].id)

[<Gene 0 at 0x7fce9445e680>, <Gene 26_AT1 at 0x7fce9445e5f0>, <Gene 314_AT1 at 0x7fce9445e590>, <Gene 8639_AT1 at 0x7fce9445e6e0>, <Gene 314_AT2 at 0x7fce9445e6b0>]
[<Gene ENSG00000000419 at 0x7fce8b9afd90>, <Gene ENSG00000001036 at 0x7fce8b9afd00>, <Gene ENSG00000001084 at 0x7fce8b9afdc0>, <Gene ENSG00000001630 at 0x7fce8b9afbb0>, <Gene ENSG00000002549 at 0x7fce8b9afc10>]
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_functional', '_id', '_model', '_reaction', '_repr_html_', '_set_id_with_model', 'annotation', 'copy', 'functional', 'id', 'knock_out', 'model', 'name', 'notes', 'reactions']
ENSG00000000419


H-gem uporablja ENS genske oznake, medtem RECON3D genske simbole.

Pripravimo si slovarja `id gena` --> `ime gena` in obratno:

In [54]:
gene_id_name = {} # seznam ID genov
gene_name_id = {} # seznam imen genov

for g in model.genes[1:]:
    gene_id_name[g.id] = g.name
    gene_name_id[g.name] = g.id

In [55]:
list(gene_id_name.items())[:5] # prvih 5 elementov v slovarju, ki ID gena poveže z imenom

[('26_AT1', 'AOC1'),
 ('314_AT1', 'AOC2'),
 ('8639_AT1', 'AOC3'),
 ('314_AT2', 'AOC2'),
 ('1591_AT1', 'CYP24A1')]

In [56]:
list(gene_name_id.items())[:5] # prvih 5 elementov v slovarju, ki ime gena poveže z ID

[('AOC1', '26_AT1'),
 ('AOC2', '314_AT2'),
 ('AOC3', '8639_AT1'),
 ('CYP24A1', '1591_AT1'),
 ('CYP27B1', '1594_AT1')]

Preberimo podatke, ki smo jih prej shranili:

In [57]:
df_control = pd.read_csv(os.path.join('data','data_normal.txt'))
df_tumor = pd.read_csv(os.path.join('data','data_tumor.txt'))

Imenom genov pripišimo še ustrezne ID-je:

In [58]:
df_control['gene_id'] = df_control['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")
df_tumor['gene_id'] = df_tumor['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")

In [59]:
df_control

,ENS_ID,gene,value,gene_id
0,ENSG00000000460,C1orf112,5.264295,
1,ENSG00000001084,GCLC,11.328790,2729_AT1
2,ENSG00000001497,LAS1L,11.233168,
3,ENSG00000001617,SEMA3F,11.262117,
4,ENSG00000002016,RAD52,9.059880,
...,...,...,...,...
9266,ENSG00000285662,FAM245B,8.201485,
9267,ENSG00000285665,TOP1P1,4.920023,
9268,ENSG00000285735,LINC02717,0.000000,
9269,ENSG00000285967,NIPBL-DT,9.908149,


Ohranimo samo tiste vnose, ki jih imamo v modelu Recon3D:

In [60]:
df_control
print("Genes successfully mapped: ", len(df_control[df_control['gene_id'] != '']) / len(df_control))

Genes successfully mapped:  0.09416459928810268


In [61]:
df_control_sub_recon = df_control[df_control['gene_id'] != ""]
df_tumor_sub_recon = df_tumor[df_tumor['gene_id'] != ""]

In [62]:
len(df_control_sub_recon), len(df_tumor_sub_recon)

(873, 873)

In [63]:
df_control_sub_recon.describe()

,value
count,873.000000
mean,9.925020
std,3.306377
min,0.000000
25%,8.530502
50%,10.722373
75%,12.141992
max,16.627521


In [64]:
df_tumor_sub_recon.describe()

,value
count,873.000000
mean,8.210470
std,3.347648
min,0.000000
25%,5.628118
50%,9.223192
75%,10.663793
max,14.523701


In v modelu H-GEM

In [65]:
# Create a list of all genes in H-GEM
model_H_gem_genes = [g.id for g in model_H_gem.genes]
model_H_gem_genes

# Subsett all df_control rows where col 'ENS_ID' is not present in model_H_gem genes id
index = df_control['ENS_ID'].isin(model_H_gem_genes)

# Get the percentage of genes that are present in H-GEM
print("Genes successfully mapped: ", len(df_control[index]) / len(df_control))


df_control_sub_H_gem = df_control[index]
df_tumor_sub_H_gem = df_tumor[index]
df_tumor_sub_H_gem

Genes successfully mapped:  0.13881997627008952


,ENS_ID,gene,value,gene_id
1,ENSG00000001084,GCLC,10.149710,2729_AT1
5,ENSG00000002549,LAP3,11.185164,51056_AT1
6,ENSG00000002726,AOC1,5.725368,26_AT1
8,ENSG00000002746,HECW1,6.364500,
10,ENSG00000003137,CYP26B1,10.147565,
...,...,...,...,...
9127,ENSG00000276043,UHRF1,12.102169,
9131,ENSG00000276293,PIP4K2B,13.246074,8396_AT1
9154,ENSG00000277494,GPIHBP1,8.294008,
9162,ENSG00000277893,SRD5A2,3.159216,6716_AT1


In [66]:
len(df_control_sub_H_gem), len(df_tumor_sub_H_gem)

(1287, 1287)

In [67]:
df_control_sub_H_gem.describe()

,value
count,1287.000000
mean,10.042734
std,3.222411
min,0.000000
25%,8.691604
50%,10.801554
75%,12.088577
max,17.827263


In [68]:
df_tumor_sub_H_gem.describe()

,value
count,1287.000000
mean,8.671716
std,3.305124
min,0.000000
25%,6.541272
50%,9.597526
75%,10.961621
max,18.913443


Who captures more DE genes?

In [70]:
print('Genes in RECON3D: ', len(df_tumor_sub_recon))
print('Genes in H-GEM: ', len(df_tumor_sub_H_gem))

Genes in RECON3D:  873
Genes in H-GEM:  1287


### Integracija transkriptomskih podatkov v model

Uporabili bomo algoritem CORDA:

* [https://resendislab.github.io/corda/](https://resendislab.github.io/corda/)
* [https://github.com/resendislab/corda](https://github.com/resendislab/corda)

Funkcija za rekonstrukcijo kot argument sprejme model in preslikavo `confidence`, ki za vsako reakcijo poda mero zaupanja, da je vsebovana v modelu:
```
Allowed confidence values are -1 (absent/do not include), 0 (unknown), 1 (low confidence), 2 (medium confidence) and 3 (high confidence).
```

`confidence` si lahko pripravimo z uporabo CORDA funkcije `reaction_confidence`, ki preslika stopnje izražanja genov (po enaki lestvici kot zgoraj) v stopnje aktivnosti reakcij z uporabo pravil GPR.


Dodatno lahko nastavimo zmožnost modela, da producira biomaso:
```
met_prod (Optional[list]): Additional metabolic targets that have to be
            achieved by the model. Can be a single object or list of objects.
            List elements can be given in various forms:
            (1) A string naming a metabolite in the model will ensure that the
            metabolite can be produced.
            For instance: met_prod = ["pi_c", "atp_c"]
            (2) A dictionary of metabolite -> int will define an irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = {"adp_c": -1, "pi_c": -1, "atp_c": 1}
            (3) A string representation of a reversible or irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = "adp_c + pi_c -> atp_c"

```
Primer priprave niza `met_prod`:
```Python
met_prod = model.reactions.BIOMASS_maintenance.build_reaction_string()
met_prod = met_prod.replace("-->", "->")
```


In [71]:
biomass = model.reactions.BIOMASS_maintenance.build_reaction_string().replace("-->", "->")
biomass

'0.50563 ala__L_c + 0.35926 arg__L_c + 0.27942 asn__L_c + 0.35261 asp__L_c + 20.7045 atp_c + 0.020401 chsterol_c + 0.011658 clpn_hs_c + 0.039036 ctp_c + 0.046571 cys__L_c + 0.27519 g6p_c + 0.326 gln__L_c + 0.38587 glu__L_c + 0.53889 gly_c + 0.036117 gtp_c + 20.6508 h2o_c + 0.12641 his__L_c + 0.28608 ile__L_c + 0.54554 leu__L_c + 0.59211 lys__L_c + 0.15302 met__L_c + 0.023315 pail_hs_c + 0.15446 pchol_hs_c + 0.055374 pe_hs_c + 0.002914 pglyc_hs_c + 0.25947 phe__L_c + 0.41248 pro__L_c + 0.005829 ps_hs_c + 0.39253 ser__L_c + 0.017486 sphmyln_hs_c + 0.31269 thr__L_c + 0.013306 trp__L_c + 0.15967 tyr__L_c + 0.053446 utp_c + 0.35261 val__L_c -> 20.6508 adp_c + 20.6508 h_c + 20.6508 pi_c'

### Preslikava izražanja genov v prisotnosti reakcij

In [72]:
from corda import reaction_confidence
from corda import CORDA

Pomagali si bomo s funkcijo, ki bo preslikala izražanje genov v vrednosti -1, 0, 1, 2 in 3 preko vnaprej definiranih pragov.

In [74]:
def val_to_score(val, thr0, thr1, thr2, thr3):
    if val < thr0:
        return -1
    elif val < thr1:
        return 0
    elif val < thr2:
        return 1
    elif val < thr3:
        return 2
    else:
        return 3

#### Parametri rekonstrukcije
Pragove bomo definirali preko odstotne aktivnosti genov v vzrocu. Obstaja veliko različnih strategij, kako se tega problema lotiti. Nekaj več o tem si lahko preberete v [članku](https://doi.org/10.1371/journal.pcbi.1007185).

In [99]:
require_biomass = True
perc0 = 25 # below this ... inactive
perc1 = 50 # else, below this ... not sure
perc2 = 75 # else, below this ...  low confidence
perc3 = 95 # else, below this ... medium confidence
# else high confidence (above perc3)

In [100]:
folder = os.path.join('models','biomass') if require_biomass else os.path.join('models','no_biomass')
folder

'models/biomass'

### Rangiranje
Za vsako skupino bomo določili lastne pragove glede na aktivnosti genov:

In [111]:
# Deffine function for confidence score
def calculate_confidence(df, perc0, perc1, perc2, perc3, model, genes='gene_id'):
    
    thr0 = np.percentile(df.value, perc0)
    thr1 = np.percentile(df.value, perc1)
    thr2 = np.percentile(df.value, perc2)
    thr3 = np.percentile(df.value, perc3)
    
    # Najprej preslikajmo intenziteto izražanja genov v skupine
    gene_conf = {g_id: val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df[genes], df['value'])}

    # Potem pa skupine genov uporabimo pri razvrščanju reakcij v skupine z uporabo
    # CORDA funkcije `reaction_confidence`, ki pri preslikavi upošteva pravila GPR.
    conf = {}
    for r in model.reactions:
        conf[r.id] = reaction_confidence(r, gene_conf)

    return conf

In [102]:
conf_control = calculate_confidence(
    df = df_control_sub_recon,
    perc0 = perc0,
    perc1 = perc1,
    perc2 = perc2,
    perc3 = perc3,
    model = model
    )
conf_control

{'24_25DHVITD3tm': 0,
 '25HVITD3t': 0,
 'COAtl': 0,
 'EX_5adtststerone_e': 0,
 'EX_5adtststerones_e': 0,
 'EX_5fthf_e': 0,
 'EX_5htrp_e': 0,
 'EX_5mthf_e': 0,
 'EX_5thf_e': 0,
 'EX_6dhf_e': 0,
 '24_25VITD3Hm': 0,
 '24NPHte': 0,
 '10FTHF7GLUtl': 0,
 '10FTHFtm': 0,
 '11DOCRTSLtr': 0,
 '13DAMPPOX': 2,
 '24_25DHVITD2t': 0,
 '24_25DHVITD2tm': 0,
 '24_25DHVITD3t': 0,
 '25VITD2Hm': 0,
 '2AMACHYD': 0,
 '2AMACSULT': 0,
 '2AMADPTm': -1,
 '2MCITt': 0,
 '2OXOADOXm': 0,
 '2OXOADPTm': -1,
 '34DHPHEt': 1,
 '34DHXMANDACOX': 1,
 '34HPLFM': 0,
 '3AIBtm': 2,
 '3HAO': 1,
 '3MOBt2im': 0,
 '3MOPt2im': 0,
 '3MOXTYROX': 2,
 '3SALAOX': 0,
 '42A12BOOX': 2,
 '4HBZCOAFm': 0,
 '4HBZFm': 0,
 '4HDEBRISOQUINEte': 0,
 '4HOXPACDOX_NADP': 1,
 '4MOPt2im': 0,
 '4MPTNLte': 0,
 '4NPHSULT': 0,
 '4PYRDX': 0,
 '5ADTSTSTERONEGLCtr': 0,
 '5ADTSTSTERONESULT': 0,
 '5ADTSTSTERONEte': 0,
 '5FTHFt2_1': 0,
 '5HOXINOXDA': 2,
 '5THFtl': 0,
 '5THFtm': 0,
 '7DHCHSTEROLtr': 0,
 '7DHFtm': 0,
 'A4GALTc': 2,
 'A4GALTg': 2,
 'A4GNT2g': 0,
 'AA

Potem poženimo integracijo z algoritmom:

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [103]:
if require_biomass:
    opt_control = CORDA(model, conf_control, met_prod = biomass)
else:
    opt_control = CORDA(model, conf_control)
opt_control.build()
print(opt_control)

build status: reconstruction complete
Inc. reactions: 1366/10601
 - unclear: 915/8139
 - exclude: 6/293
 - low and medium: 203/1927
 - high: 242/242



Model pretvorimo v COBRA zapis:

In [104]:
model_control = opt_control.cobra_model()

In mu dodajmo še biomaso:

In [105]:
if require_biomass:
    model_control.add_reactions([model.reactions.BIOMASS_maintenance])
    model_control.objective=model.objective
require_biomass

True

Model lahko zdaj shranimo v SBML zapis:

In [107]:
cobra.io.write_sbml_model(model_control, os.path.join(f'{folder}','model_control_RECON3D.xml'))

**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [108]:
model_control = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_control_RECON3D.xml'))

In [106]:
model_control.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
HC00250_e,EX_HC00250_e,1000,0,0.00%
HC00900_e,EX_HC00900_e,1000,4,3.79%
atp_e,EX_atp_e,26.37,10,0.25%
chsterol_e,EX_chsterol_e,5.916,27,0.15%
cmp_e,EX_cmp_e,500,9,4.26%
cpppg1_e,EX_cpppg1_e,1.705,36,0.06%
crm_hs_e,EX_crm_hs_e,5.071,19,0.09%
dtmp_e,EX_dtmp_e,1000,10,9.48%
fdp_e,EX_fdp_e,579.8,6,3.30%
gua_e,EX_gua_e,10.47,5,0.05%


Shranimo zgornje ukaze kot funkcijo:

In [133]:
def process_opt(model, conf, require_biomass, biomass=None, filename=None, folder=None, reaction = 'BIOMASS_maintenance'):
    react = getattr(model.reactions, reaction)
    react
    
    # integracijo z algoritmom
    if require_biomass:
        opt = CORDA(model, conf, met_prod=biomass)
    else:
        opt = CORDA(model, conf)

    opt.build()
    print(opt)

    # Model pretvorimo v COBRA zapis
    model_cobra = opt.cobra_model()

    # In mu dodajmo še biomaso
    if require_biomass:
        model_cobra.add_reactions([react])
        model_cobra.objective = model.objective

    # Model lahko zdaj shranimo v SBML zapis
    if filename is not None:
        cobra.io.write_sbml_model(model_cobra, os.path.join(f'{folder}', f'{filename}.xml'))
        
    return model_cobra

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [110]:
conf_tumor = calculate_confidence(
    df = df_tumor_sub_recon,
    perc0 = perc0,
    perc1 = perc1,
    perc2 = perc2,
    perc3 = perc3,
    model = model
    )

model_tumor = process_opt(
    model = model,
    conf = conf_tumor,
    require_biomass = require_biomass,
    biomass = biomass,
    filename = 'model_tumor_RECON3D',
    folder = folder
)

build status: reconstruction complete
Inc. reactions: 1777/10601
 - unclear: 1217/8002
 - exclude: 6/361
 - low and medium: 295/1979
 - high: 259/259



**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [112]:
model_tumor = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_tumor_RECON3D.xml'))

In [113]:
model_tumor.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
thmtp_c,DM_thmtp_c,1000,12,19.81%
HC00250_e,EX_HC00250_e,962.6,0,0.00%
HC01361_e,EX_HC01361_e,1000,9,14.85%
atp_e,EX_atp_e,1000,10,16.50%
chsterol_e,EX_chsterol_e,5.927,27,0.26%
crm_hs_e,EX_crm_hs_e,5.08,19,0.16%
fdp_e,EX_fdp_e,580,6,5.74%
h2o2_e,EX_h2o2_e,408.8,0,0.00%
inost_e,EX_inost_e,506.8,6,5.02%
o2_e,EX_o2_e,418.8,0,0.00%


### Rangiranje H-GEM

Control:


In [134]:
# id="MAR09931" name="Biomass maintenance reaction without replication precursors"

biomass = model_H_gem.reactions.MAR09931.build_reaction_string().replace("-->", "->")
biomass

conf_control_H_gem = calculate_confidence(
    df = df_control_sub_H_gem,
    perc0 = perc0,
    perc1 = perc1,
    perc2 = perc2,
    perc3 = perc3,
    model = model_H_gem,
    genes = 'ENS_ID'
    )

model_control_H_gem = process_opt(
    model = model_H_gem,
    conf = conf_control_H_gem,
    require_biomass = require_biomass,
    biomass = biomass,
    filename = 'model_control_H-GEM',
    folder = folder,
    reaction = 'MAR09931'
)

build status: reconstruction complete
Inc. reactions: 2835/13074
 - unclear: 2108/9465
 - exclude: 17/369
 - low and medium: 362/2874
 - high: 348/366



In [ ]:
model_control_H_gem.summary()

Zaradi dolgega časa izvajanja je debugiranje težavno.

Model summary je poročal:
No objective coefficients in model. Unclear what should be optimized

In [144]:
model_H_gem.reactions.MAR09931.build_reaction_string().replace("-->", "->")

'0.50563 MAM01307c + 0.35926 MAM01365c + 0.27942 MAM01369c + 0.35261 MAM01370c + 20.7045 MAM01371c + 0.020401 MAM01450c + 0.011658 MAM01589c + 0.039036 MAM01623c + 0.046571 MAM01628c + 0.27519 MAM01968c + 0.38587 MAM01974c + 0.326 MAM01975c + 0.53889 MAM01986c + 0.036117 MAM02034c + 20.6508 MAM02040c + 0.12641 MAM02125c + 0.28608 MAM02184c + 0.54554 MAM02360c + 0.59211 MAM02426c + 0.15302 MAM02471c + 0.15446 MAM02684c + 0.055374 MAM02685c + 0.002914 MAM02715c + 0.25947 MAM02724c + 0.023315 MAM02750c + 0.41248 MAM02770c + 0.005829 MAM02808c + 0.39253 MAM02896c + 0.017486 MAM02908c + 0.31269 MAM02993c + 0.013306 MAM03089c + 0.15967 MAM03101c + 0.053446 MAM03130c + 0.35261 MAM03135c --> 20.6508 MAM01285c + 20.6508 MAM02039c + 20.6508 MAM02751c + MAM03970c'

In [143]:
model = model_H_gem
conf = conf_control_H_gem
require_biomass = require_biomass
biomass = biomass
filename = 'model_control_H-GEM'
folder = folder
reaction = 'MAR09931'

react = getattr(model.reactions, reaction)
react



# Model lahko zdaj shranimo v SBML zapis
#if filename is not None:
 #   cobra.io.write_sbml_model(model_cobra, os.path.join(f'{folder}', f'{filename}.xml'))

conf

{'MAR03905': 0,
 'MAR03907': 2,
 'MAR04097': 1,
 'MAR04099': 2,
 'MAR04108': 1,
 'MAR04133': 1,
 'MAR04281': 0,
 'MAR04388': 0,
 'MAR04283': 1,
 'MAR08357': 3,
 'MAR04379': 3,
 'MAR04301': 3,
 'MAR04355': 3,
 'MAR04358': 1,
 'MAR04360': 0,
 'MAR04363': 0,
 'MAR04365': 0,
 'MAR04368': 3,
 'MAR04370': 0,
 'MAR04371': 0,
 'MAR04372': 0,
 'MAR04373': 0,
 'MAR04375': 3,
 'MAR04377': 1,
 'MAR04381': 3,
 'MAR04391': 0,
 'MAR04394': 0,
 'MAR04396': 1,
 'MAR04521': 1,
 'MAR06412': 0,
 'MAR07745': 2,
 'MAR07747': 0,
 'MAR08360': 0,
 'MAR08652': 1,
 'MAR08757': -1,
 'MAR03989': 2,
 'MAR04122': 0,
 'MAR04837': 0,
 'MAR05395': 0,
 'MAR05396': 0,
 'MAR09727': 1,
 'MAR05397': 0,
 'MAR05398': 0,
 'MAR05399': 0,
 'MAR05400': 0,
 'MAR05401': 0,
 'MAR08568': 0,
 'MAR08569': 0,
 'MAR08570': 0,
 'MAR08571': 0,
 'MAR08572': 0,
 'MAR08573': 0,
 'MAR08574': 0,
 'MAR08575': 0,
 'MAR08576': 0,
 'MAR08577': 0,
 'MAR08578': 0,
 'MAR08579': 0,
 'MAR08581': 0,
 'MAR08591': 2,
 'MAR08592': 2,
 'MAR08580': 0,
 'MAR08

In [145]:
# integracijo z algoritmom
#if require_biomass:
#    opt = CORDA(model, conf, met_prod=biomass)
#else:
#    opt = CORDA(model, conf)
opt = CORDA(model, conf)

opt.build()
print(opt)

# Model pretvorimo v COBRA zapis
model_cobra = opt.cobra_model()

# In mu dodajmo še biomaso
#if require_biomass:
#    model_cobra.add_reactions([react])
#    model_cobra.objective = model.objective

model_cobra.summary()

In [139]:
model_control_H_gem2 = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_control_H-GEM.xml'))

No objective coefficients in model. Unclear what should be optimized


Tumor:

In [ ]:
conf_tumor_H_gem = calculate_confidence(
    df = df_tumor_sub_H_gem,
    perc0 = perc0,
    perc1 = perc1,
    perc2 = perc2,
    perc3 = perc3,
    model = model_H_gem,
    genes = 'ENS_ID'
    )

model_tumor_H_gem = process_opt(
    model = model_H_gem,
    conf = conf_tumor_H_gem,
    require_biomass = require_biomass,
    biomass = biomass,
    filename = 'model_tumor_H-GEM',
    folder = folder,
    reaction = 'MAR09931'
)

In [ ]:
model_tumor_H_gem = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_tumor_H-GEM.xml'))

In [ ]:
model_tumor_H_gem.summary()